# Functions 

## Learning Objectives

- Define and call functions in Julia 
- Understand the use of arguments and return values in Julia functions 
- Use anonymous (lambda) functions for short snippets of functionality 
- Implement multiple dispatch by defining functions that handle different types of inputs
- Organise code into functions for clarity and reuse, and recognise the benefits of multiple dispatch for flexibility and performance

## Defining Functions

In Julia, you define a function using the `function` keyword (and terminate with `end`) or using a shorter one-line syntax. Functions are objects in Julia, and you can assign them to variables, pass them as arguments, etc.

A simple function definition would be: 
```Julia 
function say_hello(name)
    println("Hello, ", name, "!")
end
```

This function, `say_hello`, takes one argument `name` and prints a greeting. You would call it like `say_hello("Alice")`.

We return values from a function by using the `return` keyword. If you omit `return`, Julia will return the **value of the last expression** in the function (much like in R). For example:  

In [1]:
# Function to square a number (explicit return)
function square(x)
    return x * x
end

# Function to cube a number (implicit return of last expression)
function cube(x)
    x
    x * x
    x * x * x   # last expression's value will be returned
end
println(square(3))
println(cube(3))

9
27


Julia also provides a concise **single-line function definition** syntax: 
```Julia 
add(a, b) = a + b
```

This defines a function `add` that returns `a+b`. It's equivalent to writing a multi-line function that returns `a+b`. You can use whichever style you find clearer; the multi-line form is helpful for longer-function bodies. 

For example, the function below is a function that can be used to calculate the area of a circle given its radius: 

In [2]:
# Calculate area of a circle with given radius
function calculate_circle_area(radius)
    area = π * radius^2   # Julia has π predefined (or use 3.14159 for Pi)
    return area
end

r = 5.0
println("Radius: ", r, ", Area: ", calculate_circle_area(r))

Radius: 5.0, Area: 78.53981633974483


Note: In the REPL or in most editors with the Julia extension (e.g. VS Code), to get the character `π` you can type a backslash, `pi`, then hit Tab. Julia will replace it with the single-character π.

The different components of a function within Julia are: 

- **Function Name**: e.g. `calculate_circle_area` in the example above. By convention, use lowercase and underscores for multi-word names. 
- **Parameters (Arguments)**: e.g. `radius`. You can optionally add a type annotation to a parameter (like `radius::Float64`) to indicate this function is for a specific type. Still, if you leave it untyped, it will accept any types that support the operations used inside. 
- **Function Body**: the code inside, which can use the parameters. In our case, we compute `area`. 
- **Return value**: the value returned by the function. In the circle example, we used `return` for clarity, but we could also write `π * radius^2` as the last line and omit `return`. 

## Exercise: Writing a function

Write a function `f_to_c(fahrenheit)` that converts a temperature from Fahrenheit to Celsius. The formula is $C = (5/9) \times (F - 32)$. Test your function with a couple of values; we should have
```julia
f_to_c(32) == 0
f_to_c(212) == 100
```

### Keyword Arguments

Similarly to Python and R, Julia functions can also accept **keyword arguments**, which are specified by a name/value pair, with a given default value. These are defined after a semicolon (`;`) in the function signature:

In [3]:
# Return a string that describes a number with given unit and precision
# Note: `round` rounds a number to a given number of decimal places
function describe(x; unit="kg", precision=2)
    formatted = round(x; digits=precision)
    return "$formatted ($unit)"
end

# Calls:
println(describe(12.3456))                         # uses defaults: "12.35 (kg)"
println(describe(12.3456; precision=3))            # "12.346 (kg)"
println(describe(12.3456; unit="meters"))          # "12.35 (meters)"
println(describe(12.3456; unit="m", precision=1))  # "12.3 (m)"

12.35 (kg)
12.346 (kg)
12.35 (meters)
12.3 (m)


Note that the general syntax for calling a function with arguments and keyword arguments is:

```julia 
fn(arg1, arg2; kwarg1=value1, kwarg2=value2)
```

Note also that
- Argument order matters: `fn(arg1, arg2)` is not the same as `fn(arg2, arg1)`
- Keyword argument order does **not** matter, because the keyword names make order unnecessary:

  ```julia
  # These are equivalent
  fn(arg; kwarg1=value1, kwarg2=value2)
  fn(arg; kwarg2=value2, kwarg1=value1)
  ```

## Abstract Types
Julia's type hierarchy allows you to annotate against *abstract* categories, which can be helpful for writing generic, yet constrained, methods: 

```julia 
# Accept any kind of integer (e.g. Int8, Int16, UInt, etc.)
h(x::Integer) = x + 1

# Accept any real number (integer or floating point)
j(x::Real) = x * 3.5

# Accept any numeric type
k(x::Number) = x ^ 2
```

- `Number` — the root of all numeric types
- `Real` — all real numbers (subtype of `Number`; includes `Integer` and `AbstractFloat`)
- `Integer` — all integer types (subtype of `Real`)
- `AbstractFloat` — all floating-point types (subtype of `Real`)

## Multiple Dispatch 

Multiple dispatch was introduced conceptually earlier; now, let's see how to implement it with functions.

In Julia, a function should really be thought of as a **collection of methods for implementing it**. We define *generic* functions which dispatch to specific methods of implementation, depending on the type of the arguments supplied. Put another way, we can define different methods for the same function name which correspond to different parameter type signatures. Julia will dispatch (choose) the method that most precisely matches the types of the actual arguments you pass. 

Julia will check the types of **all** the arguments of a function call when determing which method to apply. This is what is meant by **multiple dispatch**: here, the 'multiple' refers to the fact that the types of all the arguments are considered, rather than e.g. just the first argument (which is what is done in R's 'generic functions', for example).

For example, imagine we want an add function that behaves differently based on argument types: 

In [4]:
# Define add for two Ints
function add(x::Int, y::Int)
    x + y
end

# Define add for two Strings (concatenate)
function add(x::String, y::String)
    return string(x, y)
end

println(add(10, 20))        
println(add("Hello", "world!"))  


30
Helloworld!


We defined two methods for `add`: one that accepts two `Int` and one that accepts two `String`. When we call `add(10, 20)`, Julia sees both arguments as `Int` and uses the integer addition method. When we call `add("Hello", "world!")`, both are `String`, so it uses the string concatenation method. The same function name, `add,` is used, but the behaviour differs by the type of arguments.

If we try to call the function `add` on a set of arguments for which there is no method defined, a `MethodError` will be thrown:

In [5]:
# Error: not defined for Int, String
add(1, "hello")

LoadError: MethodError: no method matching add(::Int64, ::String)

[0mClosest candidates are:
[0m  add([91m::String[39m, ::String)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[4]:7[24m[39m
[0m  add(::Int64, [91m::Int64[39m)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[4]:2[24m[39m


In [6]:
# Error: not defined for more than two Ints
add(1, 2, 3)

LoadError: MethodError: no method matching add(::Int64, ::Int64, ::Int64)

[0mClosest candidates are:
[0m  add(::Int64, ::Int64)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[4]:2[24m[39m


Let's now implement extra methods to handle these cases. We'll handle the case where a user can supply and `Int` and a `String`: our function `add` will convert the integer to a string and then use string concatenation.

In [7]:
# Add an Int and a String (using one-line method definition)
add(x::Int, y::String) = add(string(x), y)

# And vice-versa
add(x::String, y::Int) = add(x, string(y))

add (generic function with 4 methods)

Notice how Julia keeps track of the number of methods we've defined for `add`.

Notice also how the above implementations utilise other methods of `add` that we defined earlier. This is a common pattern: it allows us to avoid repeating similar code, which makes it easier to maintain. It also implements `add` for these mixed types on a more conceptual level: *first convert the integer to a string, then use whatever method we defined for adding two strings together*.

### Exercise: adding more methods

Add methods to handle the cases where
- A single `Int` or `String` is provided
- Three `Int`s are provided (performs usual arithmetic addition)
- Three `String`s are provided (performs string concatenation)

Try to use previously implementated methods of `add` where possible.

### Extending to handle more general types

How could we extend `add` to handle more types? It is clearly impractical to keep adding more methods on concrete types (for example, adding an `add(x::Float64, y::Float64)` or a mixed `add(x::Int, y::Float64)` etc.) Let's make the decision that, for any two objects `x` and `y`, the function `add` should perform usual numerical addition if `x` and `y` are both numbers, otherwise they should be converted to strings and concatenated. Julia makes this simple to do. We define a method for `add` that works on any two objects as follows:

In [8]:
function add(x, y)
    return string(x, y)  # converts x and y to strings then concatenates
end

add (generic function with 5 methods)

Note that this is equivalent to specifying
```julia
function add(x::Any, y::Any)
    return string(x, y)  # converts x and y to strings then concatenates
end
```

Let's now evaluate `add` on some different combinations of object. As mentioned briefly above, the key thing to remember is

> **When applying a function, Julia will use the method whose type signature most precisely matches (i.e. is most 'specialized' to) the types of the given arguments.**

Consider the following calls to `add`:

In [9]:
# String, NamedTuple
println("(String, NamedTuple): ", add("foo", (a = 1, b = 2)))

# Int, Array
println("(Int, Array): ", add(1, [2, 3]))

# Int, Int
println("(Int, Int): ", add(1, 2))

(String, NamedTuple): foo(a = 1, b = 2)
(Int, Array): 1[2, 3]
(Int, Int): 3


Notice that the method that got evaluated for `(Int, Int)` was the one we defined earlier:
```julia
function add(x::Int, y::Int)
    return x + y
end
```
since this is more specialized to the types of the arguments than the general `add(::Any, ::Any)` version we defined a moment ago.

In contrast, notice what happens if we supply an `Int` and a `Float64`:

In [10]:
# Int, Float64
println("(Int, Float64): ", add(1, 2.2))

(Int, Float64): 12.2


The version of `add` that gets applied is the `add(::Any, ::Any)` version, because there isn't a more specialized method available. To fix this, let's recall that all the numerical types defined in Julia are (concrete) subtypes of the abstract type `Number`. Julia knows how to evaluate `a + b` where `a` and `b` are instances of `Number`. We can therefore define 

In [12]:
# Add for any two numbers
function add(x::Number, y::Number)
    return x + y
end

add (generic function with 6 methods)

This method is specialized to the case where `x` and `y` are subtypes of `Number`, so now `add` will perform numerical addition whenever we supply two numbers (be they `Int`, `Float64`, `Complex`, etc.)

In [13]:
# Int, Float64 (again)
println("(Int, Float64): ", add(1, 2.2))

(Int, Float64): 3.2


### Listing methods

You can list out all methods defined for a function with the `methods` function:

In [14]:
methods(add)

# 6 methods for generic function "add" from Main:
 [1] add(x::String, y::Int64)
     @ In[7]:5
 [2] add(x::Int64, y::String)
     @ In[7]:2
 [3] add(x::String, y::String)
     @ In[4]:7
 [4] add(x::Int64, y::Int64)
     @ In[4]:2
 [5] add(x::Number, y::Number)
     @ In[12]:2
 [6] add(x, y)
     @ In[8]:1

#### Exercise: summing up

Knowing what you now know about Julia's dispatch mechanism, what's the minimum number of methods you would need to define if we were to implement `add(x, y)` again from scratch (for two arguments only)?

#### Exercise: extension to 3 args

Joe Bloggs is looking to extend `add` to handle 3 arguments. He claims "Ah, this is easy, all we need to do is define
```julia
add(x, y, z) = add(add(x, y), z)
```
and we're done!" Is Joe correct? Why / why not?

TODO: finish changes this episode

### Why Multiple Dispatch Matters 

Julia's standard library often provides a rich set of methods for functions to handle different types. Allowing functions to handle a range of argument types allows us to build up flexible and general code rather quickly: it's not uncommon to define a new function in Julia that is implemented with other functions in such a way that the new function automatically support a large variety of different argument types 'for free'.

The benefits of this are:

- **Flexibility**: You generically write code but provide special cases for specific types when needed. 
- **Performance**: Julia will pick the most specific method and compile optimised machine code for that method. 
- **Extensibility**: Users can add new methods to functions (even those from the standard library or other packages) for new types they define without modifying the original code, a form of polymorphism.

It's important to remember that there is **no dispatch on keywords**. Julia's multiple-dispatch mechanism examines **only** the type of **positional** arguments. 





In Julia, **every function is by default generic**; you can add methods to it. This is different from single-dispatch object-oriented languages (where typically a method belongs to one class and dispatch is only on the object instance type). Multiple dispatch is particularly powerful in mathematical code, where operations might naturally be defined by any combinations of operand types (e.g., mixing units, numeric types, etc.)

To summarise, multiple dispatch allows us to **write clearer code** (we don't need long chains of type-checking `if-else` inside a single function). The Julia compiler **ensures it runs fast** by picking the precise method and compiling it. 

## Anonymous Functions

Sometimes, you need a small throwaway function to pass as an argument to another function. Julia allows creating **anonymous functions** using the -> syntax. These are similar to the "lambda" function in Python.

For example, suppose we have a list of numbers, and we want to square each of them. We could write a one-liner anonymous function for squaring: `x -> x * x`. Here, `x` is the input, and `x * x` is the output. We can pass this to a higher-order function like `map`, which applies a function to each element of a collection: 

In [ ]:
numbers = [1, 2, 3, 4, 5]
squared_numbers = map(x -> x * x, numbers)
println(squared_numbers)   

In this example, `map(f, collection)` returns a new collection where function `f` has been applied to every element. We provided `map` with an anonymous function `x -> x * x`. The result `squared_numbers` is a new array of squares. 

Anonymous functions are handy for short operations, so you don't necessarily need to give a name. If the transformation is straightforward, you could also use broadcast or comprehension, but `map` with a lambda is clear and functional-style. 

You can also assign an anonymous function to a variable if you want to reuse it: 


In [ ]:
squarer = x -> x^2
println(squarer(10))

But if you're going to name it, a standard function definition might be clearer. So typically, we use the `->` syntax inline when passing to other functions or for short-lived usage. 

**Note on anonymous functions**: Unlike named functions, anonymous functions cannot have multiple methods added to them; each anonymous function carries exactly one method. If you need multiple dispatch (i.e., different behaviours for different argument types), define a standard function with `function … end` and type‐annotated methods instead.

### Exercise: Anonymous Functions 
Using an anonymous function, create an array `evens` that contains only the even numbers from an existing array `vals = [1,2,34,8,11,14]`. Hint: You might want to use the filter function!

## Mutating Functions & the `!` Naming Convention


In Julia, any function that **mutates** (i.e. changes) one or more of its arguments should have a name ending in `!`. This "bang" signals to users that the function will perform in-place modification rather than returning a new object.

**Why use `!`?**: It makes code more readable and safer by clearly distinguishing destructive operations from pure ones.

**Common examples:**
```julia
arr = [3, 1, 2]

sort!(arr)           # mutates `arr` to become [1, 2, 3]
push!(arr, 4)        # appends 4 to `arr`
pop!(arr)            # removes and returns the last element
append!(arr, [5,6])  # concatenates another collection into `arr`
```

## Organising Code with Functions 
It's good practice to wrap logic inside functions rather than writing everything in global code. Functions: 
- make code reusable (you can call the same code with different inputs easily)
- clarify intent (a function name can describe what the code does) 
- In Julia, functions are essential for performance (code inside functions is optimised and JIT-compiled, whereas code in global scope is more complex for the compiler to optimise. 
As you build larger programs, you'll likely have many small functions, each handling a specific task, which together solve your problem.

# End of Section Quiz

In [ ]:
using JSON

function show_quiz_from_json(path)
    quiz_data = JSON.parsefile(path)

    html = """
    <style>
    .quiz-question {
        background-color: #6c63ff;
        color: white;
        padding: 12px;
        border-radius: 10px;
        font-weight: bold;
        font-size: 1.2em;
        margin-bottom: 10px;
    }

    .quiz-form {
        margin-bottom: 20px;
    }

    .quiz-answer {
        display: block;
        background-color: #f2f2f2;
        border: none;
        border-radius: 10px;
        padding: 10px;
        margin: 5px 0;
        font-size: 1em;
        cursor: pointer;
        text-align: left;
        transition: background-color 0.3s;
        width: 100%;
    }

    .quiz-answer:hover {
        background-color: #e0e0e0;
    }

    .correct {
        background-color: #4CAF50 !important;
        color: white !important;
        border: none;
    }

    .incorrect {
        background-color: #D32F2F !important;
        color: white !important;
        border: none;
    }

    .feedback {
        margin-top: 10px;
        font-weight: bold;
        font-size: 1em;
    }
    </style>

    <script>
    function handleAnswer(qid, aid, feedback, isCorrect) {
        // Reset all buttons for the question
        let buttons = document.querySelectorAll(".answer-" + qid);
        buttons.forEach(btn => {
            btn.classList.remove('correct', 'incorrect');
        });

        // Apply correct/incorrect to selected
        let selected = document.getElementById(aid);
        selected.classList.add(isCorrect ? 'correct' : 'incorrect');

        // Show feedback below the question
        let feedbackBox = document.getElementById('feedback_' + qid);
        feedbackBox.innerHTML = feedback;
        feedbackBox.style.color = isCorrect ? 'green' : 'red';
    }
    </script>
    """

    for (i, question) in enumerate(quiz_data)
        qid = "$i"
        html *= """<div class="quiz-question">$(question["question"])</div><form class="quiz-form">"""

        for (j, answer) in enumerate(question["answers"])
            aid = "q$(i)_a$(j)"
            feedback = answer["feedback"]
            correct = startswith(lowercase(feedback), "correct")
            html *= """
            <button type="button" class="quiz-answer answer-$qid" id="$aid"
                onclick="handleAnswer('$qid', '$aid', '$feedback', $(correct))">
                $(answer["answer"])
            </button>
            """
        end

        html *= """<div class="feedback" id="feedback_$qid"></div></form><hr>"""
    end

    display("text/html", html)
end


# Use the function
show_quiz_from_json("questions/summary_functions.json")